In diesem Skript wird nicht auf ein sequentielles Model (Sequential) gesetzt, sondern auf ein Model

In [20]:
import os
import numpy as np
import shutil
import tensorflow as tf
from typing import Tuple

from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

from tf_utils.Cifar10DataAdvanced import CIFAR10

In [21]:
np.random.seed(0)
tf.random.set_seed(0)

In [22]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/Cifar/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [23]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    dense_layer_size: int
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(units=dense_layer_size)(x)
    x = Activation('relu')(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [24]:
data = CIFAR10()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()

epochs = 30
batch_size = 256

param_grid={
    "optimizer": [Adam, RMSprop],
    "learning_rate": [0.001],
    "filter_block_1": [32],
    "kernel_size_block_1": [3, 5],
    "filter_block_2": [32, 64],
    "kernel_size_block_2": [3, 5],
    "filter_block_3": [64, 128],
    "kernel_size_block_3": [7],
    "dense_layer_size": [512]
}

results = {
    'best_score': -np.inf,
    'best_params': {},
    'val_scores': [],
    'params': []
}

grid = ParameterGrid(param_grid)
print(f'Parameter combinations in total: {len(grid)}')



Parameter combinations in total: 32


In [25]:
for idx, comb in enumerate(grid):
    print(f'Running combination: {idx}')

    model = build_model(
        data.img_shape,
        data.num_classes,
        **comb
    )

    model_log_dir = os.path.join(LOGS_DIR, f'modelGrid{idx}')
    if os.path.exists(model_log_dir):
        shutil.rmtree(model_log_dir)
        os.mkdir(model_log_dir)

    tb_callback = TensorBoard(
        log_dir=model_log_dir,
        histogram_freq=0,
        profile_batch=0
    )

    model.fit(
        train_dataset,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        validation_data=val_dataset,
        callbacks=[tb_callback]
    )

    val_accuracy = model.evaluate(
        val_dataset,
        batch_size=batch_size,
        verbose=0
    )[1]
    results['val_scores'].append(val_accuracy)
    results['params'].append(comb)

Running combination: 0


ValueError: Unrecognized arguments in `TensorBoard` Callback: {'print_batch'}. Supported kwargs are: {'embeddings_data', 'embeddings_layer_names', 'write_grads', 'batch_size'}

In [ ]:
best_run_idx = np.argmax(results['val_scores'])
results['best_score'] = results['val_scores'][best_run_idx]
results['best_params'] = results['params'][best_run_idx]

In [ ]:
print(f"Best score: {results['best_score']} using params: {results['best_params']}\n")

In [ ]:
scores = results['val_scores']
params = results['params']

for score, param in zip(scores, params):
    print(f'Score: {score} with param: {param}')